In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
from scipy.stats import mannwhitneyu
import matplotlib.cm
sns.set_context('talk')

import itertools
import scipy.stats
import statsmodels.stats.multitest
import statannotations.Annotator 
import matplotlib as mpl
# pd.options.display.max_rows = 300
pd.options.display.max_columns = 100
sns.set_theme(style="white")
sns.set(font_scale=1.2)
pd.options.display.max_rows = 300

from itertools import combinations
from statannotations.Annotator import Annotator
sns.set_theme(style="white")

In [ ]:
data = pd.read_csv("04_internal.csv", index_col=0)

#the indeterminate cases were the 4 few times not sent cell count but were culture negative, recode 
data.loc[data['Episode_etiology']=='Indeterminate','Episode_etiology'] = 'Micro-negative'
#rename culture negative to micro-negative
data.loc[data['Episode_etiology']=='Culture-negative','Episode_etiology'] = 'Micro-negative'
#label NPC episodes 
data.loc[data.Episode_category=='NPC','Episode_etiology']= 'Non-Pneumonia'
#recode micro-neg to bacterial ?
data.loc[data['Episode_etiology']=='Micro-negative','Episode_etiology'] = 'Bacterial'

In [ ]:
#labeling each  day 
data['was_neutropenic_1500']=np.where(data.Neutrophils<1.5,True,False)
data['was_neutropenic_1000']=np.where(data.Neutrophils<1.0,True,False)
data['was_neutropenic_500']=np.where(data.Neutrophils<0.5,True,False)

In [ ]:
# summarize features over admission

WBC_overadmission = data.groupby('patient').agg({
    'Neutrophils':'median', 
    'WBC_count':'median', 
    'was_neutropenic_1500':'max',
    'was_neutropenic_1000':'max',
    'was_neutropenic_500':'max',
    'BAL_pct_neutrophils':'median'
    }).reset_index().rename(
    columns={
        'Neutrophils':'Neutrophils_median_overadmission', 
        'WBC_count': 'WBC_count_median_overadmission',
        'was_neutropenic_1500':'was_neutropenic_1500_overadmission',
        'was_neutropenic_1000':'was_neutropenic_1000_overadmission',
        'was_neutropenic_500':'was_neutropenic_500_overadmission',
        'BAL_pct_neutrophils':'bal_pct_neutro_median_overadmission'    
        })
data = pd.merge(data, WBC_overadmission, how='left', on='patient')

## Creating Not immunocomp, immunocomp w/o neutropenia & neutropenic BY DAY

In [ ]:
def annotate_immunocomp_today(row):
    if row.was_neutropenic_1500 == True:
        return 'Neutropenic'
    elif row.Immunocompromised_flag == True and row.was_neutropenic_1500 == False:
        return 'Immunocompromised without neutropenia'
    elif row.Immunocompromised_flag == False:
        return 'Not immunocompromised'
    else:
        return 'Other'
    
data['immunocomp_today'] = [annotate_immunocomp_today(row) for index,row in data.iterrows()]
data.immunocomp_today.value_counts()

## Creating Not immunocomp, immunocomp w/o neutropenia & neutropenic BY ADMISSION

In [ ]:
def annotate_immunocomp_admission(row):
    if row.was_neutropenic_1500_overadmission == True:
        return 'Neutropenic during admission'
    elif row.Immunocompromised_flag == True and row.was_neutropenic_1500_overadmission == False:
        return 'Immunocompromised without neutropenia during admission'
    elif row.Immunocompromised_flag == False:
        return 'Not immunocompromised and no neutropenia during admission'
    else:
        return 'Other'
    
data['immunocomp_admission'] = [annotate_immunocomp_admission(row) for index,row in data.iterrows()]
data.immunocomp_admission.value_counts()

In [ ]:
data.drop_duplicates(subset='patient').immunocomp_admission.value_counts()

## Types of immunocomp

In [ ]:
# breaking out types of immunocompromise 

def create_additional_columns(df):
    # Initialize new columns with False
    df['solid_organ_transplant'] = False
    df['stem_cell_transplant'] = False
    df['acute_leukemia'] = False
    df['chemotherapy'] = False
    
    # Check for 'Solid organ transplant' and 'Acute leukemia' in 'type_immunocomp' column
    df['solid_organ_transplant'] = df['type_immunocomp'].str.contains('Solid organ transplant', case=False)
    df['stem_cell_transplant'] = df['type_immunocomp'].str.contains('Stem cell transplant', case=False)
    df['acute_leukemia'] = df['type_immunocomp'].str.contains('Acute leukemia', case=False)
    df['chemotherapy'] = df['type_immunocomp'].str.contains('Myelosuppressive chemotherapy', case=False)

    return df

# Apply the function to the 'dem' DataFrame
data = create_additional_columns(data)

data[['solid_organ_transplant', 'stem_cell_transplant', 'acute_leukemia', 'chemotherapy']]=data[[
    'solid_organ_transplant', 'stem_cell_transplant', 'acute_leukemia', 'chemotherapy']].fillna(False)

In [ ]:
#label whether episode was bacterial/mixed
data['had_bacterial_episode'] = np.where(data.Episode_etiology.isin(['Bacterial', 'Bacterial/viral']),1,0)

#summarise over admission
had_bacterial_episode_during_admission = data.groupby('patient').had_bacterial_episode.max().reset_index().rename(columns={'had_bacterial_episode':'had_bacterial_episode_during_admission'})

#merge back to dataframe
data = pd.merge(data, had_bacterial_episode_during_admission, how='left', on='patient')

## Table One

In [ ]:
from tableone import TableOne, load_dataset
import matplotlib.pyplot as plt

In [ ]:
single = data.drop_duplicates(subset='patient')

In [ ]:
mytable = TableOne(single, columns=['Age','Gender','Immunocompromised_flag',
                                    'solid_organ_transplant',
       'stem_cell_transplant', 'acute_leukemia', 'chemotherapy',
        'WBC_count_median_overadmission', 'Neutrophils_median_overadmission','bal_pct_neutro_median_overadmission', 'had_bacterial_episode_during_admission',
         'Cumulative_ICU_days','Binary_outcome'], 
                   
        categorical=['Gender','Immunocompromised_flag', 'solid_organ_transplant',
       'stem_cell_transplant', 'acute_leukemia', 'chemotherapy','had_bacterial_episode_during_admission','Binary_outcome'], 
                   
        nonnormal=['Age','Cumulative_ICU_days','WBC_count_median_overadmission', 
                   'Neutrophils_median_overadmission','bal_pct_neutro_median_overadmission'],
                   groupby='immunocomp_admission',#pval=True,
                overall=False, missing=False,
        rename={
            'Immunocompromised_flag' : 'Immunocompromised',
            'solid_organ_transplant' : 'Solid Organ Transplant',
            'stem_cell_transplant' : 'Stem Cell Transplant',
            'acute_leukemia' : 'Acute Leukemia',
            'chemotherapy' : 'Chemotherapy',
            'WBC_count_median_overadmission' : 'WBC Count',
            'Neutrophils_median_overadmission' : 'Neutrophil Count',
            'bal_pct_neutro_median_overadmission' : 'BAL % Neutrophils',
            'Cumulative_ICU_days' : 'Cumulative ICU Days',
            'Binary_outcome' : 'Unfavorable Outcome',
            'was_neutropenic_1500_overadmission' : 'Neutropenic',
               }
                )
mytable.to_csv('Final Table One.csv')
mytable

# BAL LEVEL DATA

In [ ]:
data['day_bucket_starts'] = pd.to_datetime(data['day_bucket_starts'], errors='coerce')
first_icu_stay_day = data.groupby(['patient']).agg({"day_bucket_starts": "min"}).rename(columns={'day_bucket_starts':'first_icu_date'})
last_icu_stay_day = data.groupby(['patient']).agg({"day_bucket_starts": "max"}).rename(columns={'day_bucket_starts':'last_icu_date'})
data = pd.merge(data, first_icu_stay_day, how='left', on='patient')
data = pd.merge(data, last_icu_stay_day, how='left', on='patient')
data['day_after_first_icu_day'] = data.day_bucket_starts - data.first_icu_date
data['day_after_first_icu_day'] = data['day_after_first_icu_day'].dt.days

data['day_bucket_starts'] = pd.to_datetime(data['day_bucket_starts'], errors='coerce')


In [ ]:
data['days_on_ventilator'] = data.groupby('patient')['Intubation_flag'].cumsum().values
data['summed_nat_score_to_today'] = data.groupby('patient')['NAT_score'].cumsum().values
data['received_abx_thisday'] = np.where(data['NAT_score'] == -2, False, True)
data['summed_days_of_abx_to_today'] = data.groupby('patient')['received_abx_thisday'].cumsum().values

In [ ]:
bals = data[data.BAL_performed==True]

first_bal = bals.drop_duplicates(subset='patient')

In [ ]:
mytable = TableOne(first_bal, columns=['day_after_first_icu_day',
                                  'days_on_ventilator','summed_days_of_abx_to_today','immunocomp_today',
                                  
                                  'BAL_pct_neutrophils','BAL_pct_lymphocytes','BAL_pct_macrophages',

                                   'Episode_etiology','Episode_category',],
                   
        categorical=['immunocomp_today','Episode_etiology','Episode_category',], 
                   
        nonnormal=['BAL_pct_neutrophils','BAL_pct_lymphocytes','BAL_pct_macrophages','day_after_first_icu_day',
#  'wbc, body fluid',
 'neutrophils, body fluid',
                                  'days_on_ventilator','summed_days_of_abx_to_today','Neutrophils'],
                 #  groupby='immunocomp_admission',#pval=True,
                  overall=False,missing=False,
        rename={
               }
                )
mytable.to_csv('BAL_first_Table_9-3-24.csv')
mytable



In [ ]:
mask = bals.index.isin(first_bal.index)

# Assign the boolean mask to the 'first_bal' column in bals
bals['first_bal'] = mask

## BAL TABLE ONE

In [ ]:
mytable = TableOne(bals, columns=['BAL_pct_neutrophils','BAL_pct_lymphocytes','BAL_pct_macrophages',
                                  'day_after_first_icu_day',
                                  'days_on_ventilator','summed_days_of_abx_to_today','Neutrophils',
                                  'immunocomp_today', 'Episode_etiology','Episode_category',],
                   
        categorical=['immunocomp_today','Episode_etiology','Episode_category',], 
                   
        nonnormal=['BAL_pct_neutrophils','BAL_pct_lymphocytes','BAL_pct_macrophages','day_after_first_icu_day',
#  'wbc, body fluid',
#  'neutrophils, body fluid',
                                  'days_on_ventilator','summed_days_of_abx_to_today','Neutrophils'],
                   groupby='first_bal',#pval=True,
                  overall=False,missing=False,
        rename={
               }
                )
mytable.to_csv('BAL Table_9-3-24.csv')
mytable


In [ ]:
bals.groupby('immunocomp_today')[['Episode_etiology']].value_counts()

## CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix

matrix = data[['immunocomp_today','BAL_pct_neutrophils','Pathogen_bacteria_detected']].dropna()
matrix['bal_pct_neutro_above_50'] = np.where(matrix['BAL_pct_neutrophils'] >= 50, True, False, )

### Bacterial/Viral

In [ ]:
#episode-defining BALs and adjudcation as gold standard 

matrix = data[['immunocomp_today','Episode_etiology','BAL_pct_neutrophils','Pathogen_bacteria_detected']].dropna()
matrix['bal_pct_neutro_above_50'] = np.where(matrix['BAL_pct_neutrophils'] >= 50, True, False, )
matrix['adjudicated_bacterial'] = np.where(matrix['Episode_etiology'].isin(['Bacterial', 'Bacterial/viral']), True, False, )


In [ ]:
#episode-defining BALs and adjudcation as gold standard 
from sklearn.metrics import ConfusionMatrixDisplay

for group in matrix['immunocomp_today'].unique():

    group_data = matrix[matrix['immunocomp_today'] == group]
    
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(group_data['adjudicated_bacterial'].astype(int),group_data['bal_pct_neutro_above_50'].astype(int), 
                                   )
    tn, fp, fn, tp = conf_matrix.ravel()
    
    # Calculate sensitivity, specificity, PPV, NPV
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)

    print(f'Group: {group}')
    # Plotting the confusion matrix as a heatmap
    plt.figure(figsize=(4, 3))
    # sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g', cbar=False,
    #             xticklabels=['0', '1'],
    #             yticklabels=['0', '1'])
    
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,)
    disp.plot(cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('BAL%PMNs>50')
    plt.ylabel('adjudicated_bacterial or bacterial/viral')
    # Adjusting tick label font size
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)
    plt.show()

    # Print the confusion matrix and metrics

    print(f'Sensitivity: {sensitivity:.2f}')
    print(f'Specificity: {specificity:.2f}')
    print(f'PPV (Precision): {ppv:.2f}')
    print(f'NPV: {npv:.2f}')
    print("")

In [ ]:
#episode-defining BALs and adjudcation as gold standard 
from sklearn.metrics import ConfusionMatrixDisplay

for group in matrix['immunocomp_today'].unique():

    group_data = matrix[matrix['immunocomp_today'] == group]
    
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(group_data['Pathogen_bacteria_detected'].astype(int),group_data['bal_pct_neutro_above_50'].astype(int), 
                                   )
    tn, fp, fn, tp = conf_matrix.ravel()
    
    # Calculate sensitivity, specificity, PPV, NPV
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)

    print(f'Group: {group}')
    # Plotting the confusion matrix as a heatmap
    plt.figure(figsize=(4, 3))
    # sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g', cbar=False,
    #             xticklabels=['0', '1'],
    #             yticklabels=['0', '1'])
    
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,)
    disp.plot(cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('BAL%PMNs>50')
    plt.ylabel('Pathogen_bacteria_detected or bacterial/viral')
    # Adjusting tick label font size
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)
    plt.show()

    # Print the confusion matrix and metrics

    print(f'Sensitivity: {sensitivity:.2f}')
    print(f'Specificity: {specificity:.2f}')
    print(f'PPV (Precision): {ppv:.2f}')
    print(f'NPV: {npv:.2f}')
    print("")

In [ ]:
data.loc[data.Episode_category=='NPC','Episode_etiology']= 'Non-Pneumonia Control'

In [ ]:
states_order = ['Neutropenic', 'Immunocompromised without neutropenia', 'Not immunocompromised']
subcat_order = ["Bacterial", "Viral", "Bacterial/viral", "Non-Pneumonia Control"]

subcat_palette = sns.color_palette("pastel",n_colors=5) #placeholder since not used
states_palette = sns.set_palette(["firebrick", "orange", "silver"])

x= 'Episode_etiology'
hue = 'immunocomp_today'

hue_plot_params = {
    'data': data,
    'x': 'Episode_etiology',
    'y': 'BAL_pct_neutrophils',
    "order": subcat_order,
    "hue": "immunocomp_today",
    "hue_order": states_order,
    "palette": states_palette
}

pair_list = []
for c in data[x].dropna().unique():
    sub_combos = list(combinations(data[hue].dropna().unique(),2))
    for combo in sub_combos:
        pair_list.append([tuple([c, combo[0]]), tuple([c, combo[1]])])
pair_list

In [ ]:
# Reorder the 'immunocomp_today' according to the desired order
states_order = ['Neutropenic', 'Immunocompromised without neutropenia', 'Not immunocompromised']
data['immunocomp_today'] = pd.Categorical(data['immunocomp_today'], categories=states_order, ordered=True)


# Figure 1A

In [ ]:
data.Episode_category.value_counts()

In [ ]:
fig, ax = plt.subplots(figsize = (15, 6))

hue_plot_params = {
    'data': data,
    'x': 'Episode_etiology',
    'y': 'Neutrophils',
    "order": subcat_order,
    "hue": "immunocomp_today",
    "hue_order": states_order,
    "palette": states_palette
}
pairs = pair_list


ax = sns.stripplot(ax=ax, **hue_plot_params, dodge=True, jitter=.2)
box = sns.boxplot(ax=ax, **hue_plot_params, showfliers=False, boxprops=dict(alpha=.3))

# Calculate number of obs per group & median to position labels
counts = data.groupby(['Episode_etiology', 'immunocomp_today']).size().reset_index(name='cnt')

for pos, label in enumerate(ax.get_xticklabels()):
    for shift, immu in enumerate(['Neutropenic', 'Immunocompromised without neutropenia', 'Not immunocompromised']):
        x_pos = pos + (shift - 1) * 0.2
        n_obs = counts.cnt[counts.Episode_etiology.eq(label.get_text()) & counts.immunocomp_today.eq(immu)].values[0]
        ax.text(x_pos, -8, f'$n={n_obs}$',
                horizontalalignment='center',
                size='x-small',
                color='black',
                weight='semibold')

annotator = Annotator(ax, pairs, **hue_plot_params)
annotator.configure(test="Mann-Whitney", verbose=2,text_format='simple',show_test_name=False, hide_non_significant=True).apply_and_annotate()
ax.set_ylim(-10)

handles, labels = ax.get_legend_handles_labels()
new_labels = ['Patients with neutropenia', 'Patients with imunocompromise on study entry', 'Patients with neither']
l = plt.legend(handles[0:3], new_labels[0:3], bbox_to_anchor=(.5, 1.15), loc='center', borderaxespad=0)

plt.title("Peripheral neutrophils by episode etiology in patients with neutropenia, immunocompromise on study entry, and neither")
plt.xlabel("Episode etiology")
plt.ylabel("Peripheral neutrophils")
fig.text(0.01, 0.98, "B", weight="bold", horizontalalignment='left', verticalalignment='center')
# sns.set_theme(style="white")
plt.show()

#Save image
fig.savefig('Figure1A_9-3-24.pdf', bbox_inches='tight')

# Figure 1B

In [ ]:

fig, ax = plt.subplots(figsize = (15, 6))

hue_plot_params = {
    'data': data,
    'x': 'Episode_etiology',
    'y': 'BAL_pct_neutrophils',
    "order": subcat_order,
    "hue": "immunocomp_today",
    "hue_order": states_order,
    "palette": states_palette
}
pairs = pair_list


ax = sns.stripplot(ax=ax, **hue_plot_params, dodge=True, jitter=.2)
box = sns.boxplot(ax=ax, **hue_plot_params, showfliers=False, boxprops=dict(alpha=.3))

# Calculate number of obs per group & median to position labels
counts = data.groupby(['Episode_etiology', 'immunocomp_today']).size().reset_index(name='cnt')

for pos, label in enumerate(ax.get_xticklabels()):
    for shift, immu in enumerate(['Neutropenic', 'Immunocompromised without neutropenia', 'Not immunocompromised']):
        x_pos = pos + (shift - 1) * 0.2
        n_obs = counts.cnt[counts.Episode_etiology.eq(label.get_text()) & counts.immunocomp_today.eq(immu)].values[0]
        ax.text(x_pos, -8, f'$n={n_obs}$',
                horizontalalignment='center',
                size='x-small',
                color='black',
                weight='semibold')

annotator = Annotator(ax, pairs, **hue_plot_params)
annotator.configure(test="Mann-Whitney", verbose=2,text_format='simple',show_test_name=False, hide_non_significant=True).apply_and_annotate()
ax.set_ylim(-10)

handles, labels = ax.get_legend_handles_labels()
new_labels = ['Patients with neutropenia', 'Patients with immunocompromise on study entry', 'Patients with neither']
l = plt.legend(handles[0:3], new_labels[0:3], bbox_to_anchor=(.5, 1.15), loc='center', borderaxespad=0)

plt.title("BAL percent neutrophils by episode etiology in in patients with neutropenia, immunocompromise on study entry, and neither")
plt.xlabel("Episode etiology")
plt.ylabel("BAL percent neutrophils")
fig.text(0.01, 0.98, "A", weight="bold", horizontalalignment='left', verticalalignment='center')
# sns.set_theme(style="white")
plt.show()

#Save image
fig.savefig('Figure1B_9-3-24.pdf', bbox_inches='tight')

# Mann-Whitney U: Not immunocomp vs. Immunocomp w/o neutro

In [ ]:
# Splitting the data into two groups 
group1 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Immunocompromised without neutropenia'))]['BAL_pct_neutrophils'].dropna()
group2 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Not immunocompromised'))]['BAL_pct_neutrophils'].dropna()

# Performing the Mann-Whitney U test
mannwhitney_result = mannwhitneyu(group1, group2)

# Calculate median and IQR for each group
median_group1 = group1.median()
q1_group1, q3_group1 = group1.quantile(0.25), group1.quantile(0.75)
median_group2 = group2.median()
q1_group2, q3_group2 = group2.quantile(0.25), group2.quantile(0.75)

# Print the results in a sentence
result_sentence = (
    "The Mann-Whitney U test revealed no significant difference in BAL percent neutrophils between \n"
    "not immunocompromised patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]) \n"
    "and immunocompromised without neutropenia patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]), \n"
    "U statistic = {:.2f}, p-value = {:.4f}."
).format(median_group1, q1_group1, q3_group1, median_group2, q1_group2, q3_group2, mannwhitney_result.statistic, mannwhitney_result.pvalue)

print(result_sentence)

# Mann-Whitney U: Not immunocomp vs. Neutropenic

In [ ]:
# Splitting the data into two groups 
group1 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Neutropenic'))]['BAL_pct_neutrophils'].dropna()
group2 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Not immunocompromised'))]['BAL_pct_neutrophils'].dropna()

# Performing the Mann-Whitney U test
mannwhitney_result = mannwhitneyu(group1, group2)

# Calculate median and IQR for each group
median_group1 = group1.median()
q1_group1, q3_group1 = group1.quantile(0.25), group1.quantile(0.75)
median_group2 = group2.median()
q1_group2, q3_group2 = group2.quantile(0.25), group2.quantile(0.75)

# Print the results in a sentence
result_sentence = (
    "The Mann-Whitney U test revealed a significant difference in BAL percent neutrophils between \n"
    "not immunocompromised patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]) \n"
    "and neutropenic patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]), \n"
    "U statistic = {:.2f}, p-value = {:.6f}."
).format(median_group1, q1_group1, q3_group1, median_group2, q1_group2, q3_group2, mannwhitney_result.statistic, mannwhitney_result.pvalue)

print(result_sentence)

# Mann-Whitney U: Immunocomp w/o neutropenia vs. Neutropenic

In [ ]:
# Splitting the data into two groups 
group1 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Neutropenic'))]['BAL_pct_neutrophils'].dropna()
group2 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Immunocompromised without neutropenia'))]['BAL_pct_neutrophils'].dropna()

# Performing the Mann-Whitney U test
mannwhitney_result = mannwhitneyu(group1, group2)

# Calculate median and IQR for each group
median_group1 = group1.median()
q1_group1, q3_group1 = group1.quantile(0.25), group1.quantile(0.75)
median_group2 = group2.median()
q1_group2, q3_group2 = group2.quantile(0.25), group2.quantile(0.75)

# Print the results in a sentence
result_sentence = (
    "The Mann-Whitney U test revealed a signifcant difference in BAL percent neutrophils between \n"
    "immunocompromised without neutropenia patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]) \n"
    "and neutropenic patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]), \n"
    "U statistic = {:.2f}, p-value = {:.6f}."
).format(median_group1, q1_group1, q3_group1, median_group2, q1_group2, q3_group2, mannwhitney_result.statistic, mannwhitney_result.pvalue)

print(result_sentence)

## Mann-Whitney U: Immunocomp vs Neither -- Peripheral Neutrophils

In [ ]:
# Splitting the data into two groups 
group1 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Immunocompromised without neutropenia'))]['Neutrophils'].dropna()
group2 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Not immunocompromised'))]['Neutrophils'].dropna()

# Performing the Mann-Whitney U test
mannwhitney_result = mannwhitneyu(group1, group2)

# Calculate median and IQR for each group
median_group1 = group1.median()
q1_group1, q3_group1 = group1.quantile(0.25), group1.quantile(0.75)
median_group2 = group2.median()
q1_group2, q3_group2 = group2.quantile(0.25), group2.quantile(0.75)

# Print the results in a sentence
result_sentence = (
    "The Mann-Whitney U test revealed no significant difference in peripheral neutrophils between \n"
    "immunocompromised on study entry (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]) \n"
    "and neither patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]), \n"
    "U statistic = {:.2f}, p-value = {:.4f}."
).format(median_group1, q1_group1, q3_group1, median_group2, q1_group2, q3_group2, mannwhitney_result.statistic, mannwhitney_result.pvalue)

print(result_sentence)

## Mann-Whitney U: Neutropenic vs Neither -- Peripheral Neutrophils

In [ ]:
# Splitting the data into two groups 
group1 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Neutropenic'))]['Neutrophils'].dropna()
group2 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Not immunocompromised'))]['Neutrophils'].dropna()

# Performing the Mann-Whitney U test
mannwhitney_result = mannwhitneyu(group1, group2)

# Calculate median and IQR for each group
median_group1 = group1.median()
q1_group1, q3_group1 = group1.quantile(0.25), group1.quantile(0.75)
median_group2 = group2.median()
q1_group2, q3_group2 = group2.quantile(0.25), group2.quantile(0.75)

# Print the results in a sentence
result_sentence = (
    "The Mann-Whitney U test revealed a significant difference in peripheral neutrophils between \n"
    "neutropenic patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]) \n"
    "and neither patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]), \n"
    "U statistic = {:.2f}, p-value = {:.6f}."
).format(median_group1, q1_group1, q3_group1, median_group2, q1_group2, q3_group2, mannwhitney_result.statistic, mannwhitney_result.pvalue)

print(result_sentence)

## Mann-Whitney U: Neutropenic vs IC -- Peripheral Neutrophils

In [ ]:
# Splitting the data into two groups 
group1 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Neutropenic'))]['Neutrophils'].dropna()
group2 = data[((data.Episode_etiology=='Bacterial') & (data.immunocomp_today=='Immunocompromised without neutropenia'))]['Neutrophils'].dropna()

# Performing the Mann-Whitney U test
mannwhitney_result = mannwhitneyu(group1, group2)

# Calculate median and IQR for each group
median_group1 = group1.median()
q1_group1, q3_group1 = group1.quantile(0.25), group1.quantile(0.75)
median_group2 = group2.median()
q1_group2, q3_group2 = group2.quantile(0.25), group2.quantile(0.75)

# Print the results in a sentence
result_sentence = (
    "The Mann-Whitney U test revealed a signifcant difference in peripheral neutrophils between \n"
    "neutropenic patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]) \n"
    "and immunocompromised patients with bacterial pneumonia (median [q1, q3]: {:.2f} [{:.2f}, {:.2f}]), \n"
    "U statistic = {:.2f}, p-value = {:.6f}."
).format(median_group1, q1_group1, q3_group1, median_group2, q1_group2, q3_group2, mannwhitney_result.statistic, mannwhitney_result.pvalue)

print(result_sentence)

# ROC Curves

In [ ]:
# ROC curves

bals['adjudicated_bacterial'] = np.where(bals['Episode_etiology'].isin(['Bacterial', 'Bacterial/viral']), 1, 0, )

roc_data = bals.loc[bals.Episode_category.isin(['VAP', 'CAP', 'HAP', 'NPC']),['immunocomp_today', 'BAL_pct_neutrophils', 'adjudicated_bacterial', 'Pathogen_bacteria_detected']].dropna()

In [ ]:
roc_data['Pathogen_bacteria_detected'] = roc_data['Pathogen_bacteria_detected'].astype(int)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve


fpr, tpr, thresh = metrics.roc_curve(roc_data['adjudicated_bacterial'], roc_data['BAL_pct_neutrophils'])
auc = metrics.auc(fpr, tpr)

fig, ax = plt.subplots(figsize = (8,6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' %auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL%PMNs', fontsize=14)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

fig.savefig('AUROC_all_adj.pdf')

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve


fpr, tpr, thresh = metrics.roc_curve(roc_data['Pathogen_bacteria_detected'], roc_data['BAL_pct_neutrophils'])
auc = metrics.auc(fpr, tpr)

fig, ax = plt.subplots(figsize = (8,6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' %auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL%PMNs (Pathogen_bacteria_detected)', fontsize=14)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

fig.savefig('AUROC_all.pdf')

## ROC Neutropenic - Adjudicated

In [ ]:
fpr, tpr, thresh = metrics.roc_curve(roc_data.loc[roc_data.immunocomp_today=='Neutropenic', 'adjudicated_bacterial'], 
                                     roc_data.loc[roc_data.immunocomp_today=='Neutropenic', 'BAL_pct_neutrophils'])
auc = metrics.auc(fpr, tpr)

fig, ax = plt.subplots(figsize = (8,6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' %auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL%PMNs', fontsize=14)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

fig.savefig('AUROC_neutropenic_adj.pdf')

## ROC Neutropenic - Pathogen Detected

In [ ]:
fpr, tpr, thresh = metrics.roc_curve(roc_data.loc[roc_data.immunocomp_today=='Neutropenic', 'Pathogen_bacteria_detected'], 
                                     roc_data.loc[roc_data.immunocomp_today=='Neutropenic', 'BAL_pct_neutrophils'])
auc = metrics.auc(fpr, tpr)

fig, ax = plt.subplots(figsize = (8,6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' %auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL%PMNs', fontsize=14)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

fig.savefig('AUROC_neutropenic.pdf')

### Bootstrapped for 95% CI

In [ ]:
#bootstrapped for 95% CI

# Calculate the ROC curve and AUC
fpr, tpr, thresh = metrics.roc_curve(
    roc_data.loc[roc_data.immunocomp_today == 'Neutropenic', 'Pathogen_bacteria_detected'], 
    roc_data.loc[roc_data.immunocomp_today == 'Neutropenic', 'BAL_pct_neutrophils']
)
auc = metrics.auc(fpr, tpr)

# Bootstrap to calculate 95% confidence intervals for AUC
n_bootstraps = 1000
rng = np.random.RandomState(seed=42)  # Set a random seed for reproducibility
bootstrapped_aucs = []

# Perform bootstrapping
for i in range(n_bootstraps):
    # Resample with replacement
    indices = rng.choice(len(fpr), size=len(fpr), replace=True)
    if len(np.unique(roc_data.loc[roc_data.immunocomp_today == 'Neutropenic', 'Pathogen_bacteria_detected'].iloc[indices])) < 2:
        # Skip this iteration if the resample does not contain both classes
        continue
    
    fpr_boot, tpr_boot, _ = metrics.roc_curve(
        roc_data.loc[roc_data.immunocomp_today == 'Neutropenic', 'Pathogen_bacteria_detected'].iloc[indices], 
        roc_data.loc[roc_data.immunocomp_today == 'Neutropenic', 'BAL_pct_neutrophils'].iloc[indices]
    )
    bootstrapped_auc = metrics.auc(fpr_boot, tpr_boot)
    bootstrapped_aucs.append(bootstrapped_auc)

# Calculate the 95% confidence interval
auc_ci_lower = np.percentile(bootstrapped_aucs, 2.5)
auc_ci_upper = np.percentile(bootstrapped_aucs, 97.5)

# Print the AUC with 95% CI
print(f"AUC: {auc:.2f} (95% CI: {auc_ci_lower:.2f} - {auc_ci_upper:.2f})")

# Plotting the ROC curve
fig, ax = plt.subplots(figsize=(8, 6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' % auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL% PMNs', fontsize=14)
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

# Save the figure
#fig.savefig('AUROC_neutropenic.pdf')


## ROC Immunocompromised on study entry - Adjudicated

In [ ]:
fpr, tpr, thresh = metrics.roc_curve(roc_data.loc[roc_data.immunocomp_today=='Immunocompromised without neutropenia', 'adjudicated_bacterial'], 
                                     roc_data.loc[roc_data.immunocomp_today=='Immunocompromised without neutropenia', 'BAL_pct_neutrophils'])
auc = metrics.auc(fpr, tpr)

fig, ax = plt.subplots(figsize = (8,6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' %auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL%PMNs', fontsize=14)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

fig.savefig('AUROC_IC_adj.pdf')

## ROC Immunocompromised on study entry - Pathogen Detected

In [ ]:
fpr, tpr, thresh = metrics.roc_curve(roc_data.loc[roc_data.immunocomp_today=='Immunocompromised without neutropenia', 'Pathogen_bacteria_detected'], 
                                     roc_data.loc[roc_data.immunocomp_today=='Immunocompromised without neutropenia', 'BAL_pct_neutrophils'])
auc = metrics.auc(fpr, tpr)

fig, ax = plt.subplots(figsize = (8,6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' %auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL%PMNs', fontsize=14)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

fig.savefig('AUROC_IC.pdf')

### Bootstrapped for 95% CI

In [ ]:
#bootstrapped for 95% CI

# Calculate the ROC curve and AUC
fpr, tpr, thresh = metrics.roc_curve(
    roc_data.loc[roc_data.immunocomp_today == 'Immunocompromised without neutropenia', 'Pathogen_bacteria_detected'], 
    roc_data.loc[roc_data.immunocomp_today == 'Immunocompromised without neutropenia', 'BAL_pct_neutrophils']
)
auc = metrics.auc(fpr, tpr)

# Bootstrap to calculate 95% confidence intervals for AUC
n_bootstraps = 1000
rng = np.random.RandomState(seed=42)  # Set a random seed for reproducibility
bootstrapped_aucs = []

# Perform bootstrapping
for i in range(n_bootstraps):
    # Resample with replacement
    indices = rng.choice(len(fpr), size=len(fpr), replace=True)
    if len(np.unique(roc_data.loc[roc_data.immunocomp_today == 'Immunocompromised without neutropenia', 'Pathogen_bacteria_detected'].iloc[indices])) < 2:
        # Skip this iteration if the resample does not contain both classes
        continue
    
    fpr_boot, tpr_boot, _ = metrics.roc_curve(
        roc_data.loc[roc_data.immunocomp_today == 'Immunocompromised without neutropenia', 'Pathogen_bacteria_detected'].iloc[indices], 
        roc_data.loc[roc_data.immunocomp_today == 'Immunocompromised without neutropenia', 'BAL_pct_neutrophils'].iloc[indices]
    )
    bootstrapped_auc = metrics.auc(fpr_boot, tpr_boot)
    bootstrapped_aucs.append(bootstrapped_auc)

# Calculate the 95% confidence interval
auc_ci_lower = np.percentile(bootstrapped_aucs, 2.5)
auc_ci_upper = np.percentile(bootstrapped_aucs, 97.5)

# Print the AUC with 95% CI
print(f"AUC: {auc:.2f} (95% CI: {auc_ci_lower:.2f} - {auc_ci_upper:.2f})")

# Plotting the ROC curve
fig, ax = plt.subplots(figsize=(8, 6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' % auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL% PMNs', fontsize=14)
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

## ROC Neither - Adjudicated

In [ ]:
fpr, tpr, thresh = metrics.roc_curve(roc_data.loc[roc_data.immunocomp_today=='Not immunocompromised', 'adjudicated_bacterial'], 
                                     roc_data.loc[roc_data.immunocomp_today=='Not immunocompromised', 'BAL_pct_neutrophils'])
auc = metrics.auc(fpr, tpr)

fig, ax = plt.subplots(figsize = (8,6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' %auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL%PMNs', fontsize=14)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

fig.savefig('AUROC_neither_adj.pdf')

## ROC Neither - Pathogen Detected

In [ ]:
fpr, tpr, thresh = metrics.roc_curve(roc_data.loc[roc_data.immunocomp_today=='Not immunocompromised', 'Pathogen_bacteria_detected'], 
                                     roc_data.loc[roc_data.immunocomp_today=='Not immunocompromised', 'BAL_pct_neutrophils'])
auc = metrics.auc(fpr, tpr)

fig, ax = plt.subplots(figsize = (8,6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' %auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL%PMNs', fontsize=14)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

fig.savefig('AUROC_neither.pdf')

### Bootstrapped for 95% CI

In [ ]:
#bootstrapped for 95% CI

# Calculate the ROC curve and AUC
fpr, tpr, thresh = metrics.roc_curve(
    roc_data.loc[roc_data.immunocomp_today == 'Not immunocompromised', 'Pathogen_bacteria_detected'], 
    roc_data.loc[roc_data.immunocomp_today == 'Not immunocompromised', 'BAL_pct_neutrophils']
)
auc = metrics.auc(fpr, tpr)

# Bootstrap to calculate 95% confidence intervals for AUC
n_bootstraps = 1000
rng = np.random.RandomState(seed=42)  # Set a random seed for reproducibility
bootstrapped_aucs = []

# Perform bootstrapping
for i in range(n_bootstraps):
    # Resample with replacement
    indices = rng.choice(len(fpr), size=len(fpr), replace=True)
    if len(np.unique(roc_data.loc[roc_data.immunocomp_today == 'Not immunocompromised', 'Pathogen_bacteria_detected'].iloc[indices])) < 2:
        # Skip this iteration if the resample does not contain both classes
        continue
    
    fpr_boot, tpr_boot, _ = metrics.roc_curve(
        roc_data.loc[roc_data.immunocomp_today == 'Not immunocompromised', 'Pathogen_bacteria_detected'].iloc[indices], 
        roc_data.loc[roc_data.immunocomp_today == 'Not immunocompromised', 'BAL_pct_neutrophils'].iloc[indices]
    )
    bootstrapped_auc = metrics.auc(fpr_boot, tpr_boot)
    bootstrapped_aucs.append(bootstrapped_auc)

# Calculate the 95% confidence interval
auc_ci_lower = np.percentile(bootstrapped_aucs, 2.5)
auc_ci_upper = np.percentile(bootstrapped_aucs, 97.5)

# Print the AUC with 95% CI
print(f"AUC: {auc:.2f} (95% CI: {auc_ci_lower:.2f} - {auc_ci_upper:.2f})")

# Plotting the ROC curve
fig, ax = plt.subplots(figsize=(8, 6))

plt.plot(fpr, tpr, label='ROC curve (area = %.2f)' % auc)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='pink', label='Random guess')
plt.title('ROC curve for BAL% PMNs', fontsize=14)
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.grid()
plt.legend(fontsize=12)
plt.show()

## Sensitivity threshold 

In [ ]:
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv
 
# Generate finer thresholds between 0 and 100 (e.g., 0.1 increments)
thresholds = np.arange(0, 101, 0.1)
 
results = []
 
for threshold in thresholds:
    y_pred = (roc_data['BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv = calculate_metrics(roc_data['Pathogen_bacteria_detected'], y_pred)
   
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })
   
    # Break once the sensitivity is close to 0.9
    if 0.89 <= sensitivity <= 0.91:
        print(f"Threshold: {threshold}, Sensitivity: {sensitivity}")
        break

### Threshold Neither group

In [ ]:
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv

# Filter for 'Neither'
roc_data_neither = roc_data[roc_data.immunocomp_today == 'Not immunocompromised']

# Generate finer thresholds between 0 and 100 (e.g., 0.1 increments)
thresholds = np.arange(0, 101, 0.1)

results = []

for threshold in thresholds:
    # Apply thresholding to the 'BAL_pct_neutrophils' for the 'Neither' group
    y_pred = (roc_data_neither['BAL_pct_neutrophils'] >= threshold).astype(int)
    # Calculate metrics for the 'Neither' group
    sensitivity, specificity, ppv, npv = calculate_metrics(
        roc_data_neither['Pathogen_bacteria_detected'], y_pred
    )

    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })

    # Break once the sensitivity is close to 0.9
    if 0.89 <= sensitivity <= 0.91:
        print(f"Threshold: {threshold}, Sensitivity: {sensitivity}")
        break


### Threshold for IC without group

In [ ]:
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv

# Filter for 'Immunocompromised without neutropenia'
roc_data_IC = roc_data[roc_data.immunocomp_today == 'Immunocompromised without neutropenia']

# Generate finer thresholds between 0 and 100 (e.g., 0.1 increments)
thresholds = np.arange(0, 101, 0.1)

results = []

for threshold in thresholds:
    # Apply thresholding to the 'BAL_pct_neutrophils' for the 'Immunocompromised without neutropenia' group
    y_pred = (roc_data_IC['BAL_pct_neutrophils'] >= threshold).astype(int)
    # Calculate metrics for the 'Immunocompromised without neutropenia' group
    sensitivity, specificity, ppv, npv = calculate_metrics(
        roc_data_IC['Pathogen_bacteria_detected'], y_pred
    )

    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })

    # Break once the sensitivity is close to 0.9
    if 0.89 <= sensitivity <= 0.91:
        print(f"Threshold: {threshold}, Sensitivity: {sensitivity}")
        break


### Threshold neutropenic

In [ ]:
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv

# Filter for 'Neutropenic'
roc_data_neutro = roc_data[roc_data.immunocomp_today == 'Neutropenic']

# Generate finer thresholds between 0 and 100 (e.g., 0.1 increments)
thresholds = np.arange(0, 71, 0.1)

results = []

for threshold in thresholds:
    # Apply thresholding to the 'BAL_pct_neutrophils' for the 'Neutropenic' group
    y_pred = (roc_data_neutro['BAL_pct_neutrophils'] >= threshold).astype(int)
    # Calculate metrics for the 'Neutropenic' group
    sensitivity, specificity, ppv, npv = calculate_metrics(
        roc_data_neutro['Pathogen_bacteria_detected'], y_pred
    )

    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })

    # Break once the sensitivity is close to 0.9
    if 0.89 <= sensitivity <= 0.91:
        print(f"Threshold: {threshold}, Sensitivity: {sensitivity}")
        break


In [ ]:
for threshold in thresholds:
    y_pred = (roc_data_neutro['BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv = calculate_metrics(
        roc_data_neutro['Pathogen_bacteria_detected'], y_pred
    )
    print(f"Threshold: {threshold}, Sensitivity: {sensitivity}")


In [ ]:
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv
 
# Generate finer thresholds between 0 and 100 (e.g., 0.1 increments)
thresholds = np.arange(0, 101, 0.1)
 
results = []
 
for threshold in thresholds:
    y_pred = (roc_data['BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv = calculate_metrics(roc_data['adjudicated_bacterial'], y_pred)
   
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })
   
    # Break once the sensitivity is close to 0.9
    if 0.89 <= sensitivity <= 0.91:
        print(f"Threshold: {threshold}, Sensitivity: {sensitivity}")
        break

In [ ]:
##Metrics -- All groups -- Adjudicated bacterial

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv

# Assuming 'roc_data' is your DataFrame with 'bal_pct_neutro' and 'adjudicated_bacterial'
# Make sure 'adjudicated_bacterial' is binary (0 or 1)

# Generate thresholds
thresholds = range(0, 101, 10)

results = []

for threshold in thresholds:
    y_pred = (roc_data['BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv = calculate_metrics(roc_data['adjudicated_bacterial'], y_pred)
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })

# Create a DataFrame from the results
metrics_df = pd.DataFrame(results)

# Display the table
print(metrics_df.to_string(index=False, float_format='{:.3f}'.format))

In [ ]:
##Metrics -- All groups -- Pathogen detected bacterial

def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv

# Assuming 'roc_data' is your DataFrame with 'bal_pct_neutro' and 'Pathogen_bacteria_detected'
# Make sure 'Pathogen_bacteria_detected' is binary (0 or 1)

# Generate thresholds
thresholds = range(0, 101, 10)

results = []

for threshold in thresholds:
    y_pred = (roc_data['BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv = calculate_metrics(roc_data['Pathogen_bacteria_detected'], y_pred)
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })

# Create a DataFrame from the results
metrics_df = pd.DataFrame(results)

# Display the table
print(metrics_df.to_string(index=False, float_format='{:.3f}'.format))

In [ ]:
##Metrics -- All --  Pathogen Detected

# Create the plot
fig, ax = plt.subplots(figsize=(8, 4))

# Plot each metric
ax.plot(metrics_df['Threshold'], metrics_df['Sensitivity'], marker='o', label='Sensitivity')
ax.plot(metrics_df['Threshold'], metrics_df['Specificity'], marker='s', label='Specificity')
# ax.plot(metrics_df['Threshold'], metrics_df['PPV'], marker='^', label='PPV')
# ax.plot(metrics_df['Threshold'], metrics_df['NPV'], marker='D', label='NPV')

# Customize the plot
ax.set_xlabel('BAL % Neutrophils Threshold', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Metrics vs Threshold for BAL % Neutrophils', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, linestyle='--', alpha=0.7)

# Set the x-axis to show all threshold values
ax.set_xticks(metrics_df['Threshold'])

# Add horizontal lines at 0.5 and 1.0 for reference
# ax.axhline(y=0.5, color='r', linestyle='--', alpha=0.3)
# ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.3)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


fig.savefig('metrics_all.pdf')

In [ ]:
#only for neutropenic days 


def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv

# Assuming 'roc_data' is your DataFrame with 'bal_pct_neutro' and 'adjudicated_bacterial'
# Make sure 'adjudicated_bacterial' is binary (0 or 1)

# Generate thresholds
thresholds = range(0, 101, 10)

results = []



                                     

for threshold in thresholds:
    y_pred = (roc_data.loc[roc_data.immunocomp_today=='Neutropenic', 'BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv = calculate_metrics(roc_data.loc[roc_data.immunocomp_today=='Neutropenic', 'adjudicated_bacterial'], y_pred)
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })

# Create a DataFrame from the results
metrics_df = pd.DataFrame(results)

# Display the table
print(metrics_df.to_string(index=False, float_format='{:.3f}'.format))




# Create the plot
fig, ax = plt.subplots(figsize=(8, 4))

# Plot each metric
ax.plot(metrics_df['Threshold'], metrics_df['Sensitivity'], marker='o', label='Sensitivity')
ax.plot(metrics_df['Threshold'], metrics_df['Specificity'], marker='s', label='Specificity')
# ax.plot(metrics_df['Threshold'], metrics_df['PPV'], marker='^', label='PPV')
# ax.plot(metrics_df['Threshold'], metrics_df['NPV'], marker='D', label='NPV')

# Customize the plot
ax.set_xlabel('Threshold', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Metrics vs Threshold for BAL % Neutrophils', fontsize=14)
fig.text(0.01, 0.98, "A", weight="bold", horizontalalignment='left', verticalalignment='center')
ax.legend(fontsize=10)
ax.grid(True, linestyle='--', alpha=0.7)

# Set the x-axis to show all threshold values
ax.set_xticks(metrics_df['Threshold'])

# Add horizontal lines at 0.5 and 1.0 for reference
# ax.axhline(y=0.5, color='r', linestyle='--', alpha=0.3)
# ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.3)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


fig.savefig('metrics_neutropenic_adj.pdf')


In [ ]:
#only for neutropenic days & pathogen detected

def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv

# Assuming 'roc_data' is your DataFrame with 'bal_pct_neutro' and 'adjudicated_bacterial'
# Make sure 'adjudicated_bacterial' is binary (0 or 1)

# Generate thresholds
thresholds = range(0, 101, 10)

results = []



                                     

for threshold in thresholds:
    y_pred = (roc_data.loc[roc_data.immunocomp_today=='Neutropenic', 'BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv = calculate_metrics(roc_data.loc[roc_data.immunocomp_today=='Neutropenic', 'Pathogen_bacteria_detected'], y_pred)
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })

# Create a DataFrame from the results
metrics_df = pd.DataFrame(results)

# Display the table
print(metrics_df.to_string(index=False, float_format='{:.3f}'.format))




# Create the plot
fig, ax = plt.subplots(figsize=(8, 4))

# Plot each metric
ax.plot(metrics_df['Threshold'], metrics_df['Sensitivity'], marker='o', label='Sensitivity')
ax.plot(metrics_df['Threshold'], metrics_df['Specificity'], marker='s', label='Specificity')
# ax.plot(metrics_df['Threshold'], metrics_df['PPV'], marker='^', label='PPV')
# ax.plot(metrics_df['Threshold'], metrics_df['NPV'], marker='D', label='NPV')

# Customize the plot
ax.set_xlabel('BALF % Neutrophils Threshold', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Metrics vs Threshold for BAL % Neutrophils - Patients with neutropenia', fontsize=14)
fig.text(0.01, 0.98, "A", weight="bold", horizontalalignment='left', verticalalignment='center')
ax.legend(fontsize=10)
ax.grid(True, linestyle='--', alpha=0.7)

# Set the x-axis to show all threshold values
ax.set_xticks(metrics_df['Threshold'])

# Add horizontal lines at 0.5 and 1.0 for reference
# ax.axhline(y=0.5, color='r', linestyle='--', alpha=0.3)
# ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.3)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


fig.savefig('metrics_neutropenic_.pdf')


In [ ]:
#Neither & pathogen detected

def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv

# Assuming 'roc_data' is your DataFrame with 'bal_pct_neutro' and 'adjudicated_bacterial'
# Make sure 'adjudicated_bacterial' is binary (0 or 1)

# Generate thresholds
thresholds = range(0, 101, 10)

results = []



                                     

for threshold in thresholds:
    y_pred = (roc_data.loc[roc_data.immunocomp_today=='Not immunocompromised', 'BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv = calculate_metrics(roc_data.loc[roc_data.immunocomp_today=='Not immunocompromised', 'Pathogen_bacteria_detected'], y_pred)
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })

# Create a DataFrame from the results
metrics_df = pd.DataFrame(results)

# Display the table
print(metrics_df.to_string(index=False, float_format='{:.3f}'.format))




# Create the plot
fig, ax = plt.subplots(figsize=(8, 4))

# Plot each metric
ax.plot(metrics_df['Threshold'], metrics_df['Sensitivity'], marker='o', label='Sensitivity')
ax.plot(metrics_df['Threshold'], metrics_df['Specificity'], marker='s', label='Specificity')
#ax.plot(metrics_df['Threshold'], metrics_df['PPV'], marker='^', label='PPV')
#ax.plot(metrics_df['Threshold'], metrics_df['NPV'], marker='D', label='NPV')

# Customize the plot
ax.set_xlabel('BALF % Neutrophils Threshold', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Metrics vs Threshold for BAL % Neutrophils - Patients with neither', fontsize=14)
fig.text(0.01, 0.98, "C", weight="bold", horizontalalignment='left', verticalalignment='center')
ax.legend(fontsize=10)
ax.grid(True, linestyle='--', alpha=0.7)

# Set the x-axis to show all threshold values
ax.set_xticks(metrics_df['Threshold'])

# Add horizontal lines at 0.5 and 1.0 for reference
# ax.axhline(y=0.5, color='r', linestyle='--', alpha=0.3)
# ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.3)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


fig.savefig('metrics_neither_.pdf')


In [ ]:
#IC & pathogen detected

def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return sensitivity, specificity, ppv, npv

# Assuming 'roc_data' is your DataFrame with 'bal_pct_neutro' and 'adjudicated_bacterial'
# Make sure 'adjudicated_bacterial' is binary (0 or 1)

# Generate thresholds
thresholds = range(0, 101, 10)

results = []



                                     

for threshold in thresholds:
    y_pred = (roc_data.loc[roc_data.immunocomp_today=='Immunocompromised without neutropenia', 'BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv = calculate_metrics(roc_data.loc[roc_data.immunocomp_today=='Immunocompromised without neutropenia', 'Pathogen_bacteria_detected'], y_pred)
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    })

# Create a DataFrame from the results
metrics_df = pd.DataFrame(results)

# Display the table
print(metrics_df.to_string(index=False, float_format='{:.3f}'.format))




# Create the plot
fig, ax = plt.subplots(figsize=(8, 4))

# Plot each metric
ax.plot(metrics_df['Threshold'], metrics_df['Sensitivity'], marker='o', label='Sensitivity')
ax.plot(metrics_df['Threshold'], metrics_df['Specificity'], marker='s', label='Specificity')
# ax.plot(metrics_df['Threshold'], metrics_df['PPV'], marker='^', label='PPV')
# ax.plot(metrics_df['Threshold'], metrics_df['NPV'], marker='D', label='NPV')

# Customize the plot
ax.set_xlabel('BALF % Neutrophils Threshold', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Metrics vs Threshold for BAL % Neutrophils - Patients with immunocompromise', fontsize=14)
fig.text(0.01, 0.98, "B", weight="bold", horizontalalignment='left', verticalalignment='center')
ax.legend(fontsize=10)
ax.grid(True, linestyle='--', alpha=0.7)

# Set the x-axis to show all threshold values
ax.set_xticks(metrics_df['Threshold'])

# Add horizontal lines at 0.5 and 1.0 for reference
# ax.axhline(y=0.5, color='r', linestyle='--', alpha=0.3)
# ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.3)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


fig.savefig('metrics_IC.pdf')


In [ ]:
#Neutropenic Operating Characteristic Table
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    
    # Positive and Negative Likelihood Ratios
    plr = sensitivity / (1 - specificity) if specificity < 1 else float('inf')
    nlr = (1 - sensitivity) / specificity if specificity > 0 else float('inf')
    
    # Confidence Intervals for PLR and NLR
    plr_ci = np.exp(np.log(plr) + np.array([-1, 1]) * 1.96 * np.sqrt((1 - sensitivity) / tp + specificity / tn)) if tp > 0 and tn > 0 else (0, 0)
    nlr_ci = np.exp(np.log(nlr) + np.array([-1, 1]) * 1.96 * np.sqrt(sensitivity / tp + (1 - specificity) / tn)) if tp > 0 and tn > 0 else (0, 0)
    
    return sensitivity, specificity, ppv, npv, plr, nlr, plr_ci, nlr_ci

# Assuming 'roc_data' is your DataFrame with 'bal_pct_neutro' and 'adjudicated_bacterial'
# Make sure 'adjudicated_bacterial' is binary (0 or 1)

# Generate thresholds
thresholds = range(0, 101, 10)

results = []

for threshold in thresholds:
    y_pred = (roc_data.loc[roc_data.immunocomp_today == 'Neutropenic', 'BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv, plr, nlr, plr_ci, nlr_ci = calculate_metrics(
        roc_data.loc[roc_data.immunocomp_today == 'Neutropenic', 'Pathogen_bacteria_detected'], 
        y_pred
    )
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv,
        'PLR': plr,
        'PLR_CI_Lower': plr_ci[0],
        'PLR_CI_Upper': plr_ci[1],
        'NLR': nlr,
        'NLR_CI_Lower': nlr_ci[0],
        'NLR_CI_Upper': nlr_ci[1]
    })

# Create a DataFrame from the results
metrics_df = pd.DataFrame(results)

# Display the table
print(metrics_df.to_string(index=False, float_format='{:.3f}'.format))

# Create the plot
fig, ax = plt.subplots(figsize=(8, 4))

# Plot each metric
ax.plot(metrics_df['Threshold'], metrics_df['Sensitivity'], marker='o', label='Sensitivity')
ax.plot(metrics_df['Threshold'], metrics_df['Specificity'], marker='s', label='Specificity')
#ax.plot(metrics_df['Threshold'], metrics_df['PPV'], marker='^', label='PPV')
#ax.plot(metrics_df['Threshold'], metrics_df['NPV'], marker='D', label='NPV')

# Customize the plot
ax.set_xlabel('Threshold', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Metrics vs Threshold for BAL % Neutrophils', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, linestyle='--', alpha=0.7)

# Set the x-axis to show all threshold values
ax.set_xticks(metrics_df['Threshold'])

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

metrics_df.to_excel('Neutropenic_OCs.xlsx', index=False)

#fig.savefig('metrics_neutropenic_.pdf')


In [ ]:
#Immunocompromised without neutropenia Operating Characteristic Table
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    
    # Positive and Negative Likelihood Ratios
    plr = sensitivity / (1 - specificity) if specificity < 1 else float('inf')
    nlr = (1 - sensitivity) / specificity if specificity > 0 else float('inf')
    
    # Confidence Intervals for PLR and NLR
    plr_ci = np.exp(np.log(plr) + np.array([-1, 1]) * 1.96 * np.sqrt((1 - sensitivity) / tp + specificity / tn)) if tp > 0 and tn > 0 else (0, 0)
    nlr_ci = np.exp(np.log(nlr) + np.array([-1, 1]) * 1.96 * np.sqrt(sensitivity / tp + (1 - specificity) / tn)) if tp > 0 and tn > 0 else (0, 0)
    
    return sensitivity, specificity, ppv, npv, plr, nlr, plr_ci, nlr_ci

# Assuming 'roc_data' is your DataFrame with 'bal_pct_neutro' and 'adjudicated_bacterial'
# Make sure 'adjudicated_bacterial' is binary (0 or 1)

# Generate thresholds
thresholds = range(0, 101, 10)

results = []

for threshold in thresholds:
    y_pred = (roc_data.loc[roc_data.immunocomp_today == 'Immunocompromised without neutropenia', 'BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv, plr, nlr, plr_ci, nlr_ci = calculate_metrics(
        roc_data.loc[roc_data.immunocomp_today == 'Immunocompromised without neutropenia', 'Pathogen_bacteria_detected'], 
        y_pred
    )
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv,
        'PLR': plr,
        'PLR_CI_Lower': plr_ci[0],
        'PLR_CI_Upper': plr_ci[1],
        'NLR': nlr,
        'NLR_CI_Lower': nlr_ci[0],
        'NLR_CI_Upper': nlr_ci[1]
    })

# Create a DataFrame from the results
metrics_df = pd.DataFrame(results)

# Display the table
print(metrics_df.to_string(index=False, float_format='{:.3f}'.format))

# Create the plot
fig, ax = plt.subplots(figsize=(8, 4))

# Plot each metric
ax.plot(metrics_df['Threshold'], metrics_df['Sensitivity'], marker='o', label='Sensitivity')
ax.plot(metrics_df['Threshold'], metrics_df['Specificity'], marker='s', label='Specificity')
#ax.plot(metrics_df['Threshold'], metrics_df['PPV'], marker='^', label='PPV')
#ax.plot(metrics_df['Threshold'], metrics_df['NPV'], marker='D', label='NPV')

# Customize the plot
ax.set_xlabel('Threshold', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Metrics vs Threshold for BAL % Neutrophils', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, linestyle='--', alpha=0.7)

# Set the x-axis to show all threshold values
ax.set_xticks(metrics_df['Threshold'])

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

metrics_df.to_excel('Immunocompromised without neutropenia_OCs.xlsx', index=False)

#fig.savefig('metrics_neutropenic_.pdf')


In [ ]:
#Not immunocompromised Operating Characteristic Table
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    
    # Positive and Negative Likelihood Ratios
    plr = sensitivity / (1 - specificity) if specificity < 1 else float('inf')
    nlr = (1 - sensitivity) / specificity if specificity > 0 else float('inf')
    
    # Confidence Intervals for PLR and NLR
    plr_ci = np.exp(np.log(plr) + np.array([-1, 1]) * 1.96 * np.sqrt((1 - sensitivity) / tp + specificity / tn)) if tp > 0 and tn > 0 else (0, 0)
    nlr_ci = np.exp(np.log(nlr) + np.array([-1, 1]) * 1.96 * np.sqrt(sensitivity / tp + (1 - specificity) / tn)) if tp > 0 and tn > 0 else (0, 0)
    
    return sensitivity, specificity, ppv, npv, plr, nlr, plr_ci, nlr_ci

# Assuming 'roc_data' is your DataFrame with 'bal_pct_neutro' and 'adjudicated_bacterial'
# Make sure 'adjudicated_bacterial' is binary (0 or 1)

# Generate thresholds
thresholds = range(0, 101, 10)

results = []

for threshold in thresholds:
    y_pred = (roc_data.loc[roc_data.immunocomp_today == 'Not immunocompromised', 'BAL_pct_neutrophils'] >= threshold).astype(int)
    sensitivity, specificity, ppv, npv, plr, nlr, plr_ci, nlr_ci = calculate_metrics(
        roc_data.loc[roc_data.immunocomp_today == 'Not immunocompromised', 'Pathogen_bacteria_detected'], 
        y_pred
    )
    results.append({
        'Threshold': threshold,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv,
        'PLR': plr,
        'PLR_CI_Lower': plr_ci[0],
        'PLR_CI_Upper': plr_ci[1],
        'NLR': nlr,
        'NLR_CI_Lower': nlr_ci[0],
        'NLR_CI_Upper': nlr_ci[1]
    })

# Create a DataFrame from the results
metrics_df = pd.DataFrame(results)

# Display the table
print(metrics_df.to_string(index=False, float_format='{:.3f}'.format))

# Create the plot
fig, ax = plt.subplots(figsize=(8, 4))

# Plot each metric
ax.plot(metrics_df['Threshold'], metrics_df['Sensitivity'], marker='o', label='Sensitivity')
ax.plot(metrics_df['Threshold'], metrics_df['Specificity'], marker='s', label='Specificity')
#ax.plot(metrics_df['Threshold'], metrics_df['PPV'], marker='^', label='PPV')
#ax.plot(metrics_df['Threshold'], metrics_df['NPV'], marker='D', label='NPV')

# Customize the plot
ax.set_xlabel('Threshold', fontsize=12)
ax.set_ylabel('Metric Value', fontsize=12)
ax.set_title('Metrics vs Threshold for BAL % Neutrophils', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, linestyle='--', alpha=0.7)

# Set the x-axis to show all threshold values
ax.set_xticks(metrics_df['Threshold'])

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

metrics_df.to_excel('Neither_OCs.xlsx', index=False)

#fig.savefig('metrics_neutropenic_.pdf')


In [ ]:
# Timeline 

In [ ]:

def plot_patient_neutrophil_timeline(data, patient):
    # Filter data for the specific patient
    patient_data = data[data['patient'] == patient].sort_values('ICU_day')
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot neutrophil counts
    ax.plot(patient_data['ICU_day'], patient_data['Neutrophils'], marker='o', linestyle='-', color='blue')
    
    # Customize the plot
    ax.set_xlabel('ICU Day', fontsize=12)
    ax.set_ylabel('Neutrophil Count', fontsize=12)
    ax.set_title(f'Neutrophil Count Timeline for Patient {patient}', fontsize=14)
    
    # Set x-axis to show integer days
    ax.set_xticks(np.arange(patient_data['ICU_day'].min(), patient_data['ICU_day'].max()+1, 1))
    
    # Add grid for better readability
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Annotate points with neutrophil values
    for i, txt in enumerate(patient_data['Neutrophils']):
        ax.annotate(f'{txt:.2f}', (patient_data['ICU_day'].iloc[i], patient_data['Neutrophils'].iloc[i]), 
                    xytext=(5, 5), textcoords='offset points')
    
    plt.tight_layout()
    plt.show()

## Counting

In [ ]:
data.BAL_performed.value_counts()

In [ ]:
data.patient.value_counts()

In [ ]:
data.drop_duplicates(subset='patient').Immunocompromised_flag.value_counts()

In [ ]:
data.drop_duplicates(subset='patient').immunocomp_admission.value_counts()

In [ ]:
data[data['immunocomp_admission'] == 'Not immunocompromised and no neutropenia during admission'].BAL_performed.value_counts()

In [ ]:
data[data['immunocomp_admission'] == 'Immunocompromised without neutropenia during admission'].BAL_performed.value_counts()

In [ ]:
data[data['immunocomp_admission'] == 'Neutropenic during admission'].BAL_performed.value_counts()

In [ ]:
data[data['immunocomp_today'] == 'Not immunocompromised'].BAL_performed.value_counts()

In [ ]:
data[data['immunocomp_today'] == 'Immunocompromised without neutropenia'].BAL_performed.value_counts()

In [ ]:
data[data['immunocomp_today'] == 'Neutropenic'].BAL_performed.value_counts()

In [ ]:
data['Episode_etiology'].value_counts()

In [ ]:
data[data['immunocomp_today'] == 'Neutropenic'].Episode_etiology.notna().sum()

In [ ]:
metrics_df.to_csv